In [2]:
import requests

year = 2023
day = 20

url = f"https://adventofcode.com/{year}/day/{day}/input"

with open("cookie.txt", "r") as file:
    session = file.read()

headers = {"cookie": f"session={session}"}

with open(f"inputs/day_{str(day).zfill(2)}_input.txt", "wb") as file:
    file.write(requests.get(url, headers=headers).content)

lines = [x.strip() for x in open(f"inputs/day_{str(day).zfill(2)}_input.txt", "r")]
print(lines[:5])

['%rp -> gq, sd', '&kh -> cs', '%jz -> pl, jb', '%dx -> tx', '%dh -> bm, sd']


In [3]:
from collections import defaultdict, deque

test = """broadcaster -> a, b, c
%a -> b
%b -> c
%c -> inv
&inv -> a""".split(
    "\n"
)

test2 = """broadcaster -> a
%a -> inv, con
&inv -> b
%b -> con
&con -> output""".split(
    "\n"
)

source = lines

modules = {}


def lcm(x, y):
    from math import gcd  # or can import gcd from `math` in Python 3

    return x * y // gcd(x, y)


for line in source:
    module_type_and_name, destinations = line.split(" -> ")
    module_type = module_type_and_name[0]
    module = module_type_and_name[1:]
    module_dict = {"type": module_type, "destinations": destinations.split(", ")}
    if module_type == "%":
        module_dict["state"] = 0
    if module_type == "&":
        module_dict["memory"] = defaultdict(int)
    modules[module] = module_dict

print(modules)

# second pass to update & inputs.
for line in source:
    module_type_and_name, destinations = line.split(" -> ")
    module_type = module_type_and_name[0]
    module = module_type_and_name[1:]
    destinations = destinations.split(", ")

    for destination in destinations:
        if destination not in modules:
            continue
        if modules[destination]["type"] == "&":
            modules[destination]["memory"][module] = 0


def send_pulse(params):
    module_name, pulse, from_node = params

    if module_name in ["kh", "lz", "tg", "hn"] and pulse == 0:
        found[module_name] = i + 1
        if len(found) == 4:
            print(found)

            lcm_run = 1
            for f in found.values():
                lcm_run = lcm(lcm_run, f)
            print(lcm_run)
            raise

    pulse_counts[pulse] += 1

    if module_name not in modules:
        return
    module = modules[module_name]
    if module["type"] == "b":
        for destination in module["destinations"]:
            Q.append([destination, pulse, module_name])
    if module["type"] == "%":
        if pulse == 0:
            module["state"] = (module["state"] + 1) % 2
            for destination in module["destinations"]:
                Q.append([destination, module["state"], module_name])
    if module["type"] == "&":
        module["memory"][from_node] = pulse
        for destination in module["destinations"]:
            if all([m == 1 for n, m in module["memory"].items()]):
                Q.append([destination, 0, module_name])
            else:
                Q.append([destination, 1, module_name])


Q = deque([("roadcaster", 0, "button")])

pulse_counts = {0: 0, 1: 0}


def press_button():
    # bit of a hack to drop the b here
    while Q:
        params = Q.popleft()
        send_pulse(params)


from tqdm import tqdm

found = {}

for i in range(0, 10000):
    press_button()
    Q.append(["roadcaster", 0, "button"])
    if i == 999:
        print(pulse_counts[0] * pulse_counts[1])

# only cs sends to rx
# kh, lz, tg, hn send to cs
# 599390055792 too low
# [3769, 3889, 3917, 4013]
# 230402300925361

{'rp': {'type': '%', 'destinations': ['gq', 'sd'], 'state': 0}, 'kh': {'type': '&', 'destinations': ['cs'], 'memory': defaultdict(<class 'int'>, {})}, 'jz': {'type': '%', 'destinations': ['pl', 'jb'], 'state': 0}, 'dx': {'type': '%', 'destinations': ['tx'], 'state': 0}, 'dh': {'type': '%', 'destinations': ['bm', 'sd'], 'state': 0}, 'zv': {'type': '&', 'destinations': ['ns', 'dx', 'hl', 'hn', 'fm'], 'memory': defaultdict(<class 'int'>, {})}, 'xb': {'type': '%', 'destinations': ['ds', 'sk'], 'state': 0}, 'hv': {'type': '%', 'destinations': ['sk', 'kr'], 'state': 0}, 'db': {'type': '%', 'destinations': ['zv', 'zz'], 'state': 0}, 'sk': {'type': '&', 'destinations': ['rg', 'hh', 'hv', 'kr', 'kh', 'zl', 'zn'], 'memory': defaultdict(<class 'int'>, {})}, 'tc': {'type': '%', 'destinations': ['jz'], 'state': 0}, 'dj': {'type': '%', 'destinations': ['ts', 'pl'], 'state': 0}, 'jk': {'type': '%', 'destinations': ['sd', 'vh'], 'state': 0}, 'fm': {'type': '%', 'destinations': ['dx', 'zv'], 'state': 0

RuntimeError: No active exception to reraise